# Creating Custom Dataset using DETR
> An approach for creating Dataset for Panoptic Segmentation using Pre Trained DETR.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
<!-- - image: images/chart-preview.png -->

# What and Why?

We at [TSAI](https://theschoolof.ai/) wanted to train a Panpotic Segmentation Model(DETR) on a custom dataset. We decided to go ahead with Construction Classes and annotated the things of around 50 classes like [grader](https://www.google.com/search?q=grader), [wheel loader](https://www.google.com/search?q=grader), [aac blocks](https://www.google.com/search?q=aac+blocks) etc.
While annotating we just annoatated the things, and left out stuff, as it would be way more tedious to annotate the stuffs like ground, grass, building etc. 

We know existing Models are very well trained on COCO dataset, and we could levrage them to predict stuff classes in our images. So we went out and decided to use pre-trained DETR for Panoptic Segmentation to perform inference on our images and get this stuffs for our images.

In total we had **10k Images** for all the classes combined, with very high imbalance, like 15 images for one class and on the other extreme 500+ images for other..

## Our Classes for Things and Stuff

### Things

We have 48 Things categories.

```python
{
    'aac_blocks': 0, 'adhesives': 1, 'ahus': 2, 'aluminium_frames_for_false_ceiling': 3,
    'chiller': 4, 'concrete_mixer_machine': 5, 'concrete_pump_(50%)': 6, 'control_panel': 7,
    'cu_piping': 8, 'distribution_transformer': 9, 'dump_truck___tipper_truck': 10,
    'emulsion_paint': 11, 'enamel_paint': 12, 'fine_aggregate': 13, 'fire_buckets': 14,
    'fire_extinguishers': 15, 'glass_wool': 16, 'grader': 17, 'hoist': 18,
    'hollow_concrete_blocks': 19, 'hot_mix_plant': 20, 'hydra_crane': 21,
    'interlocked_switched_socket': 22, 'junction_box': 23, 'lime': 24, 'marble': 25,
    'metal_primer': 26, 'pipe_fittings': 27, 'rcc_hume_pipes': 28, 'refrigerant_gas': 29,
    'river_sand': 30, 'rmc_batching_plant': 31, 'rmu_units': 32, 'sanitary_fixtures': 33,
    'skid_steer_loader_(bobcat)': 34, 'smoke_detectors': 35, 'split_units': 36,
    'structural_steel_-_channel': 37, 'switch_boards_and_switches': 38, 'texture_paint': 39,
    'threaded_rod': 40, 'transit_mixer': 41, 'vcb_panel': 42, 'vitrified_tiles': 43,
    'vrf_units': 44, 'water_tank': 45, 'wheel_loader': 46, 'wood_primer': 47
}
```

### Stuff

To make life simpler, I decided to make the stuff categories smaller, by collapsing all the categories to their super categories, and finally leavins us with 16 stuff classes

```python
{
    'building':48, 'ceiling':49, 'floor':50, 'food':51, 'furniture':52,
    'ground':53, 'plant':54, 'raw_material':55, 'sky':56, 'solids':57,
    'structural':58, 'textile':59, 'wall':60, 'water':61, 'window':62,
    'thing':63
}
```

Mapping for each stuff category of COCO to their super category: 

#collapse-hide
```python
{
    'cardboard': 'raw_material', 'paper': 'raw_material', 'plastic': 'raw_material',
    'metal': 'raw_material', 'wall-tile': 'wall', 'wall-panel': 'wall',
    'wall-wood': 'wall', 'wall-brick': 'wall', 'wall-stone': 'wall',
    'wall-concrete': 'wall', 'wall-other': 'wall', 'ceiling-tile': 'ceiling',
    'ceiling-other': 'ceiling', 'carpet': 'floor', 'floor-tile': 'floor',
    'floor-wood': 'floor', 'floor-marble': 'floor', 'floor-stone': 'floor',
    'floor-other': 'floor', 'window-blind': 'window', 'window-other': 'window',
    'door-stuff': 'furniture', 'desk-stuff': 'furniture', 'table': 'furniture',
    'shelf': 'furniture', 'cabinet': 'furniture', 'cupboard': 'furniture',
    'mirror-stuff': 'furniture', 'counter': 'furniture', 'light': 'furniture',
    'stairs': 'furniture', 'furniture-other': 'furniture', 'rug': 'textile',
    'mat': 'textile', 'towel': 'textile', 'napkin': 'textile', 'clothes': 'textile',
    'cloth': 'textile', 'curtain': 'textile', 'blanket': 'textile', 'pillow': 'textile',
    'banner': 'textile','textile-other': 'textile', 'fruit': 'food', 'salad': 'food',
    'vegetable': 'food', 'food-other': 'food', 'house': 'building', 'skyscraper': 'building',
    'bridge': 'building', 'tent': 'building', 'roof': 'building', 'building-other': 'building',
    'fence': 'structural', 'cage': 'structural', 'net': 'structural', 'railing': 'structural', 
    'structural-other': 'structural', 'grass': 'plant', 'tree': 'plant', 'bush': 'plant',
    'leaves': 'plant', 'flower': 'plant', 'branch': 'plant', 'moss': 'plant', 'straw': 'plant',
    'plant-other': 'plant', 'clouds': 'sky', 'sky-other': 'sky', 'wood': 'solids',
    'rock': 'solids', 'stone': 'solids', 'mountain': 'solids', 'hill': 'solids',
    'solid-other': 'solids', 'sand': 'ground', 'snow': 'ground', 'dirt': 'ground',
    'mud': 'ground', 'gravel': 'ground', 'road': 'ground', 'pavement': 'ground',
    'railroad': 'ground', 'platform': 'ground', 'playingfield': 'ground', 
    'ground-other': 'ground', 'fog': 'water', 'river': 'water', 'sea': 'water',
    'waterdrops': 'water', 'water-other': 'water',
    'things': 'things', 'water': 'water', 'window': 'window', 'ceiling': 'ceiling',
    'sky': 'sky', 'floor': 'floor', 'food': 'food', 'building': 'building','wall': 'wall'
    }
```

In [ ]:
#collapse-hide
OUR_CLASSES = {
    'aac_blocks': 0, 'adhesives': 1, 'ahus': 2, 'aluminium_frames_for_false_ceiling': 3,
    'chiller': 4, 'concrete_mixer_machine': 5, 'concrete_pump_(50%)': 6, 'control_panel': 7,
    'cu_piping': 8, 'distribution_transformer': 9, 'dump_truck___tipper_truck': 10,
    'emulsion_paint': 11, 'enamel_paint': 12, 'fine_aggregate': 13, 'fire_buckets': 14,
    'fire_extinguishers': 15, 'glass_wool': 16, 'grader': 17, 'hoist': 18,
    'hollow_concrete_blocks': 19, 'hot_mix_plant': 20, 'hydra_crane': 21,
    'interlocked_switched_socket': 22, 'junction_box': 23, 'lime': 24, 'marble': 25,
    'metal_primer': 26, 'pipe_fittings': 27, 'rcc_hume_pipes': 28, 'refrigerant_gas': 29,
    'river_sand': 30, 'rmc_batching_plant': 31, 'rmu_units': 32, 'sanitary_fixtures': 33,
    'skid_steer_loader_(bobcat)': 34, 'smoke_detectors': 35, 'split_units': 36,
    'structural_steel_-_channel': 37, 'switch_boards_and_switches': 38, 'texture_paint': 39,
    'threaded_rod': 40, 'transit_mixer': 41, 'vcb_panel': 42, 'vitrified_tiles': 43,
    'vrf_units': 44, 'water_tank': 45, 'wheel_loader': 46, 'wood_primer': 47
    }


# Annotations

In [ ]:
#collapse-hide
I = Image.open(<image_dir>/'images'/img['file_name'])
I = I.convert('RGB')

## Actual Image
![ActualImage](../images/CustomDataset/Actual.png)

In [ ]:
#collapse-hide

# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['grader']);
imgIds = coco.getImgIds(catIds=catIds );
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0] #Random Image

annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)

## Our Class Annotation (Segmentation and BBox)
![ActualAnnotation](../images/CustomDataset/ActualAnnotation.png)

In [ ]:
#collapse-hide
temp_img = np.zeros((og_w,og_h))
for op in og_poly:
    cv2.fillPoly(temp_img, pts = np.int32([op.get_xy()]), color =(255))
plt.imshow(temp_img)

![AnnotationMask](../images/CustomDataset/AnnotationMask.png)

In [ ]:
# hide
import torch
improt 